# Instance Compare

Aims to answer the question **Are there any unique indicators to malicious samples in terms of specific API Call(s) alone?** in **4.2.6. Dataset Analysis** of the study.

Note that this notebook only makes use of the verified xxxx_SampleHash_Common.csv file which represents a significant majority of the entire Oliveira dataset.  

## Import Libraries/Datasets

In [8]:
import pandas as pd
import time

malicious_df = pd.read_csv('../Clustering/(EDITED)KMeans_SampleHash_Common.csv', low_memory=False) #This should point to a verified <DataClustering>_SampleHash_Common.csv file
benign_df = pd.read_csv('../Clustering/Benign/API_Patterns.csv') #This should point to the API_Patterns.csv file

#Load list of API calls
API_LIST = "../api_calls.txt"
DELIMITER = "NaN"
API_FILE = open(API_LIST,"r")
APIS = API_FILE.readline().split(',')
# APIS.append(DELIMITER) #serves as a label for NaN values for Instance-based datasets
API_FILE.close()

## DataFrame Preview

In [9]:
def list_to_str(ls:list):
    output = ""
    for l in ls:
        output += str(l) + " "
    return output[0:len(output)-1]

def inject_patterns(inner_df:pd.DataFrame):
    patterns = []
    for row in range(inner_df.shape[0]):
        patterns.append(list_to_str(inner_df.iloc[row,2:5].transpose().to_list()))
    inner_df['type_pattern'] = patterns
    return inner_df

malicious_df.replace(to_replace='-',value='_', inplace=True)
malicious_df.drop(malicious_df[(malicious_df['Type 1']=='_')&(malicious_df['Type 2']=='_')&(malicious_df['Type 3']=='_')].index, inplace=True) #Drop row that is falsely labelled.
malicious_df = inject_patterns(malicious_df)

print("Malicious DF")
display(malicious_df)

print("Benign DF")
display(benign_df)

Malicious DF


,cluster,hash,Type 1,Type 2,Type 3,pattern,type_pattern
0,0,490d584c7d303ed35c673460b63f3ca8,trojan,dropper,pua,"GetSystemTimeAsFileTime,NtCreateMutant,GetSyst...",trojan dropper pua
1,0,9ab8ea1d2d68a0d4110df413e677976c,trojan,hacktool,_,"GetSystemTimeAsFileTime,NtCreateMutant,GetSyst...",trojan hacktool _
2,0,adbc74815ef2bd1ea4967abad812233d,trojan,_,_,"GetSystemTimeAsFileTime,NtCreateMutant,GetSyst...",trojan _ _
4,0,f5a0ad49337ebc87897698e70d03364e,trojan,dropper,_,"GetSystemTimeAsFileTime,NtCreateMutant,GetSyst...",trojan dropper _
5,0,4c972b447659f1e86769eb43593fd2a5,trojan,downloader,dropper,"GetSystemTimeAsFileTime,NtCreateMutant,GetSyst...",trojan downloader dropper
...,...,...,...,...,...,...,...
1755,198,0226e311ed2648ff399c7902fc113421,adware,trojan,_,"GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProce...",adware trojan _
1756,198,d24b78bd73f17379ed62e4c776b4f66e,trojan,adware,_,"GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProce...",trojan adware _
1757,198,f666dd4b3a53b7fe71f8976fa09bfdfb,trojan,adware,_,"GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProce...",trojan adware _
1759,199,18bce1a594550daf8b3f318de48c1674,trojan,dropper,_,"SetErrorMode,OleInitialize,LdrGetDllHandle,Ldr...",trojan dropper _


Benign DF


,hash,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,pattern
0,5b51d65972a349f90a86984c26b12b30,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,NtClose,NtQueryDirectoryFile,NtClose,LdrGetProcedureAddress,CoCreateInstance,NtOpenSection,CreateDirectoryW,NtCreateFile,LdrGetProcedureAddress,"SetErrorMode,OleInitialize,LdrGetDllHandle,Ldr..."
1,ceb8cc125478fad641daa4e04e9b2f19,GetSystemInfo,NtAllocateVirtualMemory,NtOpenSection,GetTempPathW,CreateDirectoryW,GetFileAttributesW,FindFirstFileExW,DeleteFileW,NtQueryDirectoryFile,...,NtClose,NtCreateMutant,NtClose,LdrGetDllHandle,LdrGetProcedureAddress,NtClose,NtCreateMutant,NtClose,NtCreateFile,"GetSystemInfo,NtAllocateVirtualMemory,NtOpenSe..."
2,f108600edf46d7c20f6acc522aeba6df,GetSystemTimeAsFileTime,NtProtectVirtualMemory,SetUnhandledExceptionFilter,GetTimeZoneInformation,GetSystemTimeAsFileTime,GetTimeZoneInformation,GetSystemTimeAsFileTime,GetTimeZoneInformation,GetSystemTimeAsFileTime,...,SetErrorMode,GetFileAttributesExW,SetErrorMode,NtAllocateVirtualMemory,SetErrorMode,GetFileAttributesExW,SetErrorMode,FindFirstFileExW,NtQueryDirectoryFile,"GetSystemTimeAsFileTime,NtProtectVirtualMemory..."
3,711be6337cb78a948f04759a0bd210ce,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,LdrGetProcedureAddress,NtAllocateVirtualMemory,LdrGetProcedureAddress,GetSystemMetrics,LdrLoadDll,LdrGetProcedureAddress,GetSystemMetrics,NtAllocateVirtualMemory,LdrLoadDll,"GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProce..."
4,6de26f67ceb1e3303b889489010f4c3f,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,NtClose,NtQueryDirectoryFile,NtClose,LdrGetProcedureAddress,GetSystemWindowsDirectoryW,LoadStringW,GetSystemWindowsDirectoryW,GetSystemDirectoryW,RegOpenKeyExW,"SetErrorMode,OleInitialize,LdrGetDllHandle,Ldr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,d282ef96a93986f89825508812958354,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,NtClose,LdrGetProcedureAddress,NtAllocateVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,LdrGetDllHandle,LdrGetProcedureAddress,LdrGetDllHandle,LdrGetProcedureAddress,"SetErrorMode,OleInitialize,LdrGetDllHandle,Ldr..."
1075,c0389d256f976044adf570f0df908953,GetSystemTimeAsFileTime,SetUnhandledExceptionFilter,GetCursorPos,SetErrorMode,FindResourceW,SetWindowsHookExW,CoInitializeEx,NtDuplicateObject,NtAllocateVirtualMemory,...,NtAllocateVirtualMemory,LdrLoadDll,LdrGetProcedureAddress,NtAllocateVirtualMemory,GetSystemMetrics,RegOpenKeyExW,NtAllocateVirtualMemory,GetSystemMetrics,NtAllocateVirtualMemory,"GetSystemTimeAsFileTime,SetUnhandledExceptionF..."
1076,20316e717de5db169aecbb67377504ce,SetUnhandledExceptionFilter,NtCreateMutant,NtAllocateVirtualMemory,NtClose,NtCreateMutant,NtClose,NtCreateMutant,NtClose,NtAllocateVirtualMemory,...,RegOpenKeyExW,RegQueryValueExW,RegCloseKey,RegOpenKeyExW,RegQueryValueExW,RegCloseKey,RegOpenKeyExW,RegQueryValueExW,RegCloseKey,"SetUnhandledExceptionFilter,NtCreateMutant,NtA..."
1077,ce945d424b93ea73fbbedf0254f6bc07,NtClose,NtOpenKey,NtQueryValueKey,NtClose,NtOpenKey,NtQueryValueKey,NtClose,LdrGetDllHandle,LdrGetProcedureAddress,...,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,FindResourceExW,LoadResource,"NtClose,NtOpenKey,NtQueryValueKey,NtClose,NtOp..."


## Extract Unique API Calls

In [10]:
str_output = ""

malicious_apis = []
for i in range(malicious_df.shape[0]): #Only allow those with 
    if not (malicious_df['Type 1'].iloc[i] == '_' and malicious_df['Type 2'].iloc[i] == '_' and malicious_df['Type 3'].iloc[i] == '_'):
        malicious_apis += malicious_df['pattern'].iloc[i].split(',')
malicious_apis = list(pd.Series(malicious_apis).unique())
str_output += f"# of Unique API Calls in Verified Malicious Samples: {len(malicious_apis)}\n"
str_output += str(malicious_apis) + "\n\n"

benign_apis = []
for i in range(benign_df.shape[0]): #Only allow those with 
    benign_apis += benign_df['pattern'].iloc[i].split(',')
benign_apis = list(pd.Series(benign_apis).unique())
str_output += f"# of Unique API Calls in Benign Samples: {len(benign_apis)}\n" 
str_output += str(benign_apis) + "\n\n"

with open('4 Unique_APICalls_MaliciousOnly.txt', 'w') as f:
    f.write(str_output)
    f.flush()
    f.close()
print(str_output)

# of Unique API Calls in Verified Malicious Samples: 154
['GetSystemTimeAsFileTime', 'NtCreateMutant', 'NtOpenKeyEx', 'NtQueryKey', 'LdrLoadDll', 'LdrGetProcedureAddress', 'RegOpenKeyExW', 'RegQueryInfoKeyW', 'RegEnumKeyExW', 'RegEnumValueW', 'RegCloseKey', 'GetFileAttributesW', 'RegQueryValueExW', 'NtOpenFile', 'NtQueryDirectoryFile', 'NtClose', 'NtQuerySystemInformation', 'NtProtectVirtualMemory', 'GetSystemDirectoryW', 'LdrGetDllHandle', 'NtOpenKey', 'NtQueryValueKey', 'NtCreateFile', 'GetFileSize', 'NtCreateSection', 'NtMapViewOfSection', 'GetSystemInfo', 'NtUnmapViewOfSection', 'NtAllocateVirtualMemory', 'NtFreeVirtualMemory', 'SetUnhandledExceptionFilter', 'CryptAcquireContextW', 'CryptCreateHash', 'CryptHashData', 'SetFilePointer', 'NtReadFile', 'CoInitializeEx', 'RegEnumKeyW', 'CreateThread', 'UuidCreate', 'RegCreateKeyExW', 'RegSetValueExW', 'CoCreateInstance', 'FindResourceW', 'SizeofResource', 'LoadResource', 'NtDelayExecution', 'EnumWindows', 'SetErrorMode', 'OleInitialize'

## Identify the Unique API Calls only found in Malicious API Calls.

In [11]:
str_output = ""

unique = []
for m in malicious_apis:
    if m not in benign_apis:
        unique.append(m)
str_output += f"No. of truly unique API Calls only found in Malicious Samples: {len(unique)} ({len(unique)/len(benign_apis)*100:.2f}% Matches API Calls of Benign Samples)\n"
str_output += f"Coverage of 'Malicious-only' API Calls to Official API Calls Oliveira.csv: {(len(unique)/len(APIS))*100:.4f}%\n"
str_output += "Unique API Calls to Verified Malicious Samples only: "+ str(unique) + "\n"

with open('4 Unique_APICalls_MaliciousOnly.txt', 'w') as f:
    f.write(str_output)
    f.flush()
    f.close()
print(str_output)

No. of truly unique API Calls only found in Malicious Samples: 9 (4.50% Matches API Calls of Benign Samples)
Coverage of 'Malicious-only' API Calls to Official API Calls Oliveira.csv: 2.9316%
Unique API Calls to Verified Malicious Samples only: ['CryptDecrypt', 'getaddrinfo', 'InternetOpenA', 'InternetConnectA', 'HttpOpenRequestA', 'NtSuspendThread', 'connect', 'GetDiskFreeSpaceW', 'SetFileTime']



**Fun Fact:** You might see on the results of item #5 that it will contain some Crypt (e.g., `CryptDecrypt`) related API calls. According to the talk of Sir Mantua (during his 4th hr. talk), it is a possible key indicator that a ransomware is in the system. This is supported by the fact that some malicious samples are ransomware as seen in the `Type 2` & `Type 3` malware types.

In [12]:
str_output = ""
for u in unique:
    str_output += u + "\n\n"
    str_output += str(malicious_df[malicious_df['pattern'].str.contains(u)]['type_pattern'].value_counts())
    str_output += "\n=====================================================================\n"

with open('4 APICalls_MalwareTypes_MaliciousOnly.txt', 'w') as f:
    f.write(str_output)
    f.flush()
    f.close()
print(str_output)

CryptDecrypt

type_pattern
trojan adware _             87
trojan adware virus         16
trojan _ _                  13
downloader adware trojan     9
trojan downloader adware     4
adware trojan _              3
trojan adware dropper        3
adware trojan virus          1
adware downloader trojan     1
trojan dropper _             1
Name: count, dtype: int64
getaddrinfo

type_pattern
adware trojan virus         12
adware trojan downloader     7
adware downloader trojan     1
Name: count, dtype: int64
InternetOpenA

type_pattern
trojan downloader adware    3
Name: count, dtype: int64
InternetConnectA

type_pattern
trojan downloader adware    3
Name: count, dtype: int64
HttpOpenRequestA

type_pattern
trojan downloader adware    3
Name: count, dtype: int64
NtSuspendThread

type_pattern
trojan _ _    10
Name: count, dtype: int64
connect

type_pattern
adware trojan virus         8
adware trojan downloader    2
Name: count, dtype: int64
GetDiskFreeSpaceW

type_pattern
trojan dropper _     

## Identify the Same API Calls found in both Malicious and Benign Samples.

*Apparently, due to the proliferation of trojan-like malware, it could be possible that there are more API Calls that can be found the same between Malicious and Benign Samples.*

In [13]:
str_output = ""
same = []
for m in malicious_apis:
    if m in benign_apis:
        same.append(m)
str_output += f"No. of API Calls in Malicious Samples that is found in API Calls in Benign Samples: {len(same)} ({len(same)/len(benign_apis)*100:.2f}% Matches API Calls of Benign Samples)\n"
str_output += f"Coverage of 'Same-to-Malicious-Benign-Samples' API Calls to Official API Calls Oliveira.csv: {(len(same)/len(APIS))*100:.4f}%\n"
str_output += "Unique API Calls to both Verified Malicious and Benign Samples: "+ str(same) + "\n"

with open('4 Unique_APICalls_MaliciousBenign.txt', 'w') as f:
    f.write(str_output)
    f.flush()
    f.close()
print(str_output)

No. of API Calls in Malicious Samples that is found in API Calls in Benign Samples: 145 (72.50% Matches API Calls of Benign Samples)
Coverage of 'Same-to-Malicious-Benign-Samples' API Calls to Official API Calls Oliveira.csv: 47.2313%
Unique API Calls to both Verified Malicious and Benign Samples: ['GetSystemTimeAsFileTime', 'NtCreateMutant', 'NtOpenKeyEx', 'NtQueryKey', 'LdrLoadDll', 'LdrGetProcedureAddress', 'RegOpenKeyExW', 'RegQueryInfoKeyW', 'RegEnumKeyExW', 'RegEnumValueW', 'RegCloseKey', 'GetFileAttributesW', 'RegQueryValueExW', 'NtOpenFile', 'NtQueryDirectoryFile', 'NtClose', 'NtQuerySystemInformation', 'NtProtectVirtualMemory', 'GetSystemDirectoryW', 'LdrGetDllHandle', 'NtOpenKey', 'NtQueryValueKey', 'NtCreateFile', 'GetFileSize', 'NtCreateSection', 'NtMapViewOfSection', 'GetSystemInfo', 'NtUnmapViewOfSection', 'NtAllocateVirtualMemory', 'NtFreeVirtualMemory', 'SetUnhandledExceptionFilter', 'CryptAcquireContextW', 'CryptCreateHash', 'CryptHashData', 'SetFilePointer', 'NtReadFi

In [7]:
for s in same:
    display(s, malicious_df[malicious_df['pattern'].str.contains(s)]['type_pattern'].value_counts()[0:10])

    str_output = ""
for s in same:
    str_output += s + "\n\n"
    str_output += str(malicious_df[malicious_df['pattern'].str.contains(s)]['type_pattern'].value_counts())
    str_output += "\n=====================================================================\n"

with open('4 APICalls_MalwareTypes_MaliciousBenign.txt', 'w') as f:
    f.write(str_output)
    f.flush()
    f.close()
print(str_output)

'GetSystemTimeAsFileTime'

type_pattern
trojan adware _             322
trojan adware pua           112
trojan _ _                   95
trojan adware virus          77
trojan adware downloader     42
downloader adware trojan     37
adware trojan virus          35
trojan dropper _             27
adware trojan downloader     24
trojan downloader adware     22
Name: count, dtype: int64

'NtCreateMutant'

type_pattern
trojan adware _             104
trojan _ _                   65
trojan dropper _             24
trojan adware virus          20
adware trojan virus          19
adware trojan pua            11
ransomware trojan _          11
trojan downloader adware     10
adware trojan _              10
adware trojan downloader     10
Name: count, dtype: int64

'NtOpenKeyEx'

...

'Module32FirstW'

type_pattern
trojan _ _             6
trojan miner _         1
trojan pua _           1
trojan ransomware _    1
trojan dropper worm    1
Name: count, dtype: int64

'Module32NextW'

type_pattern
trojan _ _             6
trojan miner _         1
trojan pua _           1
trojan ransomware _    1
trojan dropper worm    1
Name: count, dtype: int64

GetSystemTimeAsFileTime

type_pattern
trojan adware _              322
trojan adware pua            112
trojan _ _                    95
trojan adware virus           77
trojan adware downloader      42
downloader adware trojan      37
adware trojan virus           35
trojan dropper _              27
adware trojan downloader      24
trojan downloader adware      22
trojan adware dropper         21
adware trojan _               17
downloader trojan adware      16
trojan miner _                15
miner trojan pua              12
adware downloader trojan      12
trojan ransomware _           10
trojan pua _                   8
trojan downloader _            6
trojan downloader pua          5
adware trojan dropper          5
trojan downloader dropper      5
adware trojan pua              4
dropper trojan _               3
trojan dropper pua             3
trojan miner downloader        2
trojan miner pua               2
adware _ _                     2
downloader _ _                 2
troja